In [1]:
%pip install --upgrade pip
%pip install pandas pandas_datareader openpyxl yfinance


   ---------------------------------------- 0.0/1.8 MB ? eta -:--:--
   ------ --------------------------------- 0.3/1.8 MB 9.9 MB/s eta 0:00:01
   ---------------------- ----------------- 1.0/1.8 MB 12.7 MB/s eta 0:00:01
   ---------------------------------------  1.8/1.8 MB 14.4 MB/s eta 0:00:01
   ---------------------------------------- 1.8/1.8 MB 11.6 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.1
    Uninstalling pip-24.1.1:
      Successfully uninstalled pip-24.1.1


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import pandas_datareader.data as pdrd
import yfinance as yf
import datetime
from dateutil.relativedelta import relativedelta
#fuso_horario_local = datetime.datetime.now().astimezone().tzinfo

In [3]:
#Carregando ativos da planilha na rede de nome ativos_lista
path = r"C:\Users\dotiw\Documents\UFMG\OneDrive\.vscode\cli\DividendsDownload\ativos_lista.xlsx"
ativos_list = list(pd.read_excel(path)["Código"].values)

In [4]:
#Inicializar variáveis para captar a data mais antiga de distribuição de dividendos
ativo_mais_antigo = None
data_dividendo_atual = datetime.datetime.now().replace(tzinfo = None)

#Iterar sobre os ativos na lista
for ativo in ativos_list:
    chamada_api = yf.Ticker(ativo+'.SA')
    dividends = chamada_api.history(period = 'max')
    #Verificar se exitem dados de dividendos para o ativo
    if not dividends.empty:
        #Econtrar a data mais antiga de dividendos
        data_inicio = dividends.index.min()
        #Remover a informação de fuso horário
        data_inicio = data_inicio.replace(tzinfo = None)
        #Verificar se o ativo e a data de início do ativo atual é menor que data de início anterior registrada
        if data_inicio < data_dividendo_atual:
            data_dividendo_mais_antigo = data_inicio
            ativo_mais_antigo = ativo

#print (ativo_mais_antigo+" - Data: "+data_dividendo_mais_antigo.strftime('%Y-%m-%d'))

In [5]:
# Criando um DataFrame vazio para armazenar os dividendos por ativo
tabela_dividendos_por_ativo = pd.DataFrame()
#Obetendo os anos de referência para o index
anos = pd.date_range(start = data_dividendo_mais_antigo, end = (datetime.datetime.now() + relativedelta(years = 1)).replace(tzinfo = None), freq = 'YE').year
#Inserindo '.SA' no final dos códigos para usar como chave de busca na biblioteca
for ativo in ativos_list:
    chamada_api = yf.Ticker(ativo+'.SA')
    #Buscando o histórico de dividendos
    dividendos = chamada_api.history(period='max')['Dividends']
    #Converter o índice para o tipo datetime
    dividendos.index = pd.to_datetime(dividendos.index)
    #Agregar os dividendos por ano
    dividendos_por_ano = dividendos.groupby(dividendos.index.year).sum()
    #Reindexando para garantir que todos os anos estejam presentes em todos ativos, mesmo quando zerados
    dividendos_por_ano = dividendos_por_ano.reindex(anos, fill_value = 0)
    #Somar os dividendos por ano (antigo)
    #dividendos_por_ano = dividends_df.resample('Y').sum()
    #Consolidar os dividendos por ano por ativo
    tabela_dividendos_por_ativo[ativo] = dividendos_por_ano
#Exportar tabela para .csv
tabela_dividendos_por_ativo.to_csv('tabela_dividendos_por_ativo.csv')